In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from datetime import datetime, timedelta, timezone
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import osimport requests
import openai 
import shutil

# URL of the sitemap
sitemap_url = 'https://visittampere.fi/sitemap_index.xml'

# Send a GET request to the sitemap URL
response = requests.get(sitemap_url)

# Parse the sitemap content with BeautifulSoup
soup = BeautifulSoup(response.content, 'xml')

# Find all <loc> tags which contain URLs
urls = [loc.get_text() for loc in soup.find_all('loc')]

# Print the first few URLs for inspection
for url in urls[:10]:
    print(url)

https://visittampere.fi/page-sitemap.xml
https://visittampere.fi/news-sitemap.xml
https://visittampere.fi/events-sitemap.xml
https://visittampere.fi/blog-sitemap.xml
https://visittampere.fi/comments-sitemap.xml
https://visittampere.fi/venues-sitemap.xml
https://visittampere.fi/venues-sitemap2.xml


In [2]:
# Load environment variables
load_dotenv()

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = 'sk-key'
DATA_PATH = 'data/news'
CHROMA_PATH = "chroma"

Let's have a look what news are on VisitTampere website for a last year.

In [3]:
# Function to fetch and extract text from a news article URL
def fetch_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Assuming the news article text is contained within <p> tags
    article_text = '\n'.join([p.get_text() for p in soup.find_all('p')])
    
    return article_text

# URL of the events sitemap
sitemap_url = 'https://visittampere.fi/news-sitemap.xml'

# Send a GET request to the sitemap URL
response = requests.get(sitemap_url)

# Parse the sitemap content with BeautifulSoup
soup = BeautifulSoup(response.content, 'xml')

# Find all <url> tags
urls = soup.find_all('url')

# Get today's date and calculate the date 365 days ago (aware datetime)
today = datetime.now(timezone.utc)
thirty_days_ago = today - timedelta(days=365)

# Filter URLs based on last modification date
news_urls = []
for url in urls:
    lastmod_tag = url.find('lastmod')
    if lastmod_tag is not None:
        lastmod = lastmod_tag.text.strip()  # Assuming <lastmod> tag exists
        lastmod_date = datetime.fromisoformat(lastmod.replace('Z', '+00:00'))  # Convert to aware datetime
        if lastmod_date >= thirty_days_ago:
            loc = url.find('loc').text.strip()
            news_urls.append(loc)

news_text = []
# Fetch and print text for each news article
for url in news_urls:
    article_text = fetch_article_text(url)
    news_text.append(article_text)
    print(f"URL: {url}")
    print(f"Article Text:\n{article_text}\n")

URL: https://visittampere.fi/ajankohtaista/
Article Text:
Täältä löydät Visit Tampereen ajankohtaiset uutiset!
Puh. 03 5656 6800
visittampere@visittampere.fi
Matkailuneuvontamme palvelee ma-pe klo 10-15 puhelimitse, sähköpostitse ja sivuston chatissa.
 
Δ

URL: https://visittampere.fi/ajankohtaista/visit-tampere-mukaan-we-speak-gay-kumppaniverkostoon/
Article Text:
Visit Tampere on liittynyt mukaan We Speak Gay -kumppaniverkostoon. Kumppaniverkosto on osa We Speak Gay -yhteisöä, jonka tarkoitus on lisätä sateenkaariviestintää suomalaisessa matkailussa. Yhteisön tavoitteena on viestiä sateenkaariystävällisestä, sateenkaari-ihmisille turvallisesta matkailusta Suomessa.
”Sateenkaariviestintä on tärkeää, koska joka kolmannessa maailman maassa homoseksuaalisuus on edelleen kriminalisoitu ja suurimmassa osassa maista se ei ole sosiaalisesti hyväksyttyä. Matkustaessaan ja sopivaa matkakohdetta valitessaan sateenkaari-ihmiset joutuvat pohtimaan sitä, miten ihmiset yrityksessä tai matkakohteess

In [4]:
# Create Document objects from news_text
documents = [Document(page_content=text) for text in news_text]

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=300,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)
print(f'Split {len(documents)} news into {len(chunks)} chunks.')

Split 18 news into 74 chunks.


In [6]:
#Create a new DB from the documents.
db = Chroma.from_documents(
    chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
)

In [7]:
# Search the DB.
query_text = 'Kerro mitä uutisia tapahtui kesäkuussa'
results = db.similarity_search_with_relevance_scores(query_text, k=3)

In [8]:
if len(results) == 0 or results[0][1] < 0.7:
    print(f'Unable to find matching results.')   

In [9]:
# Define the prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

# Join the context text with separators
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

print(prompt)

# Initialize the model and generate a response
model = ChatOpenAI()
response_text = model.invoke(prompt).content

# Split the response into lines
news_items = response_text.split('\n- ')
news_items = [item for item in news_items if item.strip() != ""]

# Format each news item with a new line
formatted_news_items = "\n- ".join(news_items)
formatted_response = f"Response:\n- {formatted_news_items}\n\n"
print(formatted_response)

Human: 
Answer the question based only on the following context:

Viikon kohokohtana on saunarauhan julistus, jonka suorittaa Pekka Paasonen, Saunamestari Kilta ry:n puheenjohtaja. Julistus tapahtuu useilla kielillä Rajaportin saunalta 8. kesäkuuta.
Lisäksi viikon aikana järjestetään monipuolisesti saunatapahtumia, kuten saunalauluja Saunatemppelissä, auringonlaskun saunaelämyksiä Lapland Hotels Arenan kattosaunalla ja saunomista terassijoogan jälkeen Periscopessa.

---

Kesäkuun ensimmäisellä viikolla, 3.-9. kesäkuuta, Tampereella ja lähiseuduilla kunnioitetaan suomalaista saunakulttuuria Soul of Sauna -teemaviikon merkeissä.  Teemaviikko tarjoaa saunayrityksille ja -yhdistyksille tilaisuuden esitellä ainutlaatuisia saunaelämyksiä. Tapahtuma houkuttelee saunoihin kotimaisten matkailijoiden ja paikallisten lisäksi kansainvälisiä medioita ja vaikuttajia.
– Maailman saunapääkaupunki ja saunat yhtenä suomalaisen onnellisuuden teemana kiinnostaa tällä hetkellä matkailijoita ja mediaa runsa